<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
#подключим необходимые библиотеки
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer 
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from catboost import CatBoostClassifier

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#загрузим данные
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
#посмотрим первые пять строк датасета
df.head(5)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
#посмотрим общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
#поиск явных дубликатов
df.duplicated().sum()

0

In [6]:
#посмотрим сколько суммарно имеется прощенных значений по каждому столбцу
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [7]:
#посмотрим размер датасета
df.shape

(159292, 3)

In [8]:
class_counts = df['toxic'].value_counts()
print(class_counts)

0    143106
1     16186
Name: toxic, dtype: int64


In [9]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
#переведем текст в нижний регистр
df['text'] = df['text'].apply(lambda x: x.lower())

In [11]:
#уберем из датасета лишние символы
import re
corpus = list(df['text'])

def clear_text(text):
    text=re.sub(r'[^a-zA-Z ]', ' ', text) 
    text = re.sub(r'\s+', ' ', text)  # Замена последовательностей пробелов на один пробел
    text = text.strip()
    text=" ".join(text.split())
    return text

In [13]:
df['clear_text'] = df['text'].apply(clear_text)

In [14]:
from tqdm.notebook import tqdm
tqdm.pandas()
# Устанавливаем необходимые компоненты NLTK
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a', 'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

In [16]:
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [21]:
%%time
def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]

# Применяем функцию lemmatize_sent к столбцу с текстом в вашем датасете
df['lemmatized_text'] = df['clear_text'].apply(lemmatize_sent)

CPU times: user 8min 58s, sys: 5.29 s, total: 9min 4s
Wall time: 9min 4s


In [22]:
# Выводим примеры лемматизированных текстовых данных
for i in range(5):
    print('Original Text:', df['text'][i])
    print('Lemmatized Text:', df['lemmatized_text'][i])
    print('---')

Original Text: explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27
Lemmatized Text: ['explanation', 'why', 'the', 'edits', 'make', 'under', 'my', 'username', 'hardcore', 'metallica', 'fan', 'be', 'revert', 'they', 'weren', 't', 'vandalism', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'vote', 'at', 'new', 'york', 'doll', 'fac', 'and', 'please', 'don', 't', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'i', 'm', 'retire', 'now']
---
Original Text: d'aww! he matches this background colour i'm seemingly stuck with. thanks.  (talk) 21:51, january 11, 2016 (utc)
Lemmatized Text: ['d', 'aww', 'he', 'match', 'this', 'background', 'colour', 'i', 'm', 'seemingly', 'stick', 'with', 'thanks', 'talk', 'january', 'utc']
---
Original Text: hey man, i'm really

In [23]:
df.head()

,Unnamed: 0,text,toxic,clear_text,lemmatized_text
0,0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, make, under, my..."
1,1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...,"[d, aww, he, match, this, background, colour, ..."
2,2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...,"[hey, man, i, m, really, not, try, to, edit, w..."
3,3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,"[more, i, can, t, make, any, real, suggestion,..."
4,4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...,"[you, sir, be, my, hero, any, chance, you, rem..."


In [24]:
df.head()

,Unnamed: 0,text,toxic,clear_text,lemmatized_text
0,0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, make, under, my..."
1,1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...,"[d, aww, he, match, this, background, colour, ..."
2,2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...,"[hey, man, i, m, really, not, try, to, edit, w..."
3,3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,"[more, i, can, t, make, any, real, suggestion,..."
4,4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...,"[you, sir, be, my, hero, any, chance, you, rem..."


In [25]:
df=df.drop('Unnamed: 0',axis=1)

Вывод 

Датасет имеет 159 292 строк и 3 столбца.

Пропуски в данных отсутствуют. Форматы данных корректные.

Дубликатов не имеется.

Проведена лемматизация и очиста текстовых данных от символов

Удален не информативный столбец Unnamed: 0


## Обучение

In [47]:
df.head()

,text,toxic,clear_text,lemmatized_text
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...,"[explanation, why, the, edits, make, under, my..."
1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...,"[d, aww, he, match, this, background, colour, ..."
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...,"[hey, man, i, m, really, not, try, to, edit, w..."
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,"[more, i, can, t, make, any, real, suggestion,..."
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...,"[you, sir, be, my, hero, any, chance, you, rem..."


In [48]:
#разделим датасет
train_data, test_valid_data = train_test_split(df, test_size=0.5, random_state=42)
valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, random_state=42)

In [49]:
train_data.head()

,text,toxic,clear_text,lemmatized_text
58623,january 2009 \n\nfine then. i'll stop.,0,january fine then i ll stop,"[january, fine, then, i, ll, stop]"
147333,noteable under point 5 of the academic notabil...,0,noteable under point of the academic notabilit...,"[noteable, under, point, of, the, academic, no..."
69872,"""here is the only thing i can find regarding u...",0,here is the only thing i can find regarding us...,"[here, be, the, only, thing, i, can, find, reg..."
1552,""" 2005 (utc)\n\nthe npov tag has been restored...",0,utc the npov tag has been restored because of ...,"[utc, the, npov, tag, have, be, restore, becau..."
29797,"i dont, however gun powder ma, would you mind ...",0,i dont however gun powder ma would you mind st...,"[i, dont, however, gun, powder, ma, would, you..."


In [50]:
# Объединяем список лемматизированных слов в строку
train_data['lemmatized_text'] = train_data['lemmatized_text'].apply(lambda x: ' '.join(x))

# Создаем векторизатор
vectorizer = TfidfVectorizer(stop_words=set(nltk_stopwords.words('english')))

# Обучаем векторизатор и преобразуем тексты в числовые векторы
X_train = vectorizer.fit_transform(train_data['lemmatized_text'])
y_train = train_data['toxic']

In [51]:
test_data['lemmatized_text'] = test_data['lemmatized_text'].apply(lambda x: ' '.join(x))

# Преобразуем тексты в векторы числовых значений
X_test = vectorizer.transform(test_data['lemmatized_text'])
y_test = test_data['toxic']

In [52]:
valid_data['lemmatized_text'] = valid_data['lemmatized_text'].apply(lambda x: ' '.join(x))

# Преобразуем тексты в векторы числовых значений
X_valid = vectorizer.transform(valid_data['lemmatized_text'])
y_valid = valid_data['toxic']

Логистическая регресиия

In [58]:
c_values = [5, 15]
best_c = None
best_score = 0.0

for c in c_values:
    model_lr = LogisticRegression(C=c, class_weight='balanced', random_state=42)
    model_lr.fit(X_train, y_train)
    y_pred = model_lr.predict(X_valid)
    score = f1_score(y_valid, y_pred)
    
    if score > best_score:
        best_score = score
        best_c = c

print("Best C:", best_c)
print("Best Score:", best_score)

Best C: 15
Best Score: 0.7517730496453899


DecisionTreeClassifier

In [55]:
%%time
#найдем наилучшую модели
best_model = None
best_result = 0
best_depth = 0
for depth in range(100, 105):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучим модель с заданной глубиной дерева
	model.fit(X_train, y_train) 
	predicted_valid = model.predict(X_valid) # получим предсказания модели
    #расчитаем качество на валидационной выборке
	result =  f1_score(y_valid, predicted_valid) # посчитаем качество модели
	if result > best_result:
		best_model = model
		best_result = result
		best_depth = depth
      
print("F1 наилучшей модели на валидационной выборке:", best_result)
print("Глубина:", best_depth)

F1 наилучшей модели на валидационной выборке: 0.7089103371100853
Глубина: 101
CPU times: user 3min 2s, sys: 141 ms, total: 3min 2s
Wall time: 3min 2s


Catboost

In [66]:
%%time

# Создаем объект модели
model_cat=CatBoostClassifier(loss_function="Logloss", iterations=25)

# Обучаем модель на тренировочных данных
model_cat.fit(X_train, y_train)

# Делаем предсказания на тестовых данных
y_pred = model_cat.predict(X_valid)

f1 = f1_score(y_valid, y_pred)
      
print("F1 CatBoost на валидационной выборке:", f1)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.5
0:	learn: 0.3395394	total: 2.07s	remaining: 49.6s
1:	learn: 0.2567778	total: 4.26s	remaining: 49s
2:	learn: 0.2340780	total: 6.43s	remaining: 47.2s
3:	learn: 0.2191258	total: 8.61s	remaining: 45.2s
4:	learn: 0.2115982	total: 10.8s	remaining: 43.3s
5:	learn: 0.2053518	total: 13.2s	remaining: 41.7s
6:	learn: 0.1995245	total: 15.4s	remaining: 39.6s
7:	learn: 0.1952661	total: 17.6s	remaining: 37.5s
8:	learn: 0.1915371	total: 20.1s	remaining: 35.7s
9:	learn: 0.1885124	total: 22.5s	remaining: 33.7s
10:	learn: 0.1843611	total: 24.8s	remaining: 31.6s
11:	learn: 0.1813306	total: 27s	remaining: 29.2s
12:	learn: 0.1769573	total: 29.2s	remaining: 26.9s
13:	learn: 0.1753038	total: 31.4s	remaining: 24.6s
14:	learn: 0.1737402	total: 33.5s	remaining: 22.3s
15:	learn: 0.1719130	total: 35.7s	remaining: 20.1s
16:	learn: 0.1701668	total: 37.8s	remaining: 17.8s
17:	learn: 0.1686278	total: 39.8s	remaining: 15.5s
18:	learn: 0.1671408	total: 41.9s	remaining: 13.2s
19:	learn: 0.1652802

Проверка на тесте

In [61]:
y_pred = model_lr.predict(X_test)
# вычисляем метрики качества на тестовых данных
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)

F1 score: 0.7531689329338557


Проведена векторизация данных, тексты преобразованы в векторы числовых значений.

Датасет разделен на обучающуюся и тестовую выборки.

Модели обучены, наилучшая - модель линейной регрессии

## Выводы

Датасет имеет 159 292 строк и 3 столбца.

Пропуски в данных отсутствуют. Форматы данных корректные.

Дубликатов не имеется.

Проведена лемматизация и очиста текстовых данных от символов

Удален не информативный столбец Unnamed: 0

Проведена векторизация данных, тексты преобразованы в векторы числовых значений.

Датасет разделен на обучающуюся и тестовую выборки.

Модели обучены на

LogisticRegression = 0.75

DecisionTreeClassifier = 0.71

CatBoostClassifier = 0.68

На тестовой выборке результат = 0.75